In [1]:
import xarray as xr
import cartopy.crs as ccrs
import numpy as np
from scipy import signal 

from datetime import datetime, timedelta

import matplotlib.pyplot as plt

from utility_programs.read_routines.GITM import auto_read as auto_read_gitm
from  utility_programs.read_routines.SAMI import auto_read as auto_read_sami
import utility_programs.filters as filters

import gc

from tqdm.auto import tqdm
import glob, os

import importlib

/glade/work/abukowski/conda-envs/SAMI3-GITM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:

t_11 = datetime(2011,11,20)
t_05 = datetime(2011, 5,20)

t_start0 = timedelta(days=1) # Delta from t_start
t_end0 = timedelta(days=2)     #  in hours



dirs = {'may':'/glade/u/home/abukowski/scratch/simstorm-20110521/postprocessed/1hr/',}
        # 'nov':'/glade/u/home/abukowski/scratch/simstorm-20111121/postprocessed/1hr/'}

In [17]:

afewtimes = [10,20,30,40,50,60,70,80]

alts = [225, 400, 660, 850]

lons = np.linspace(0,360,7)[:-1]

gitmcol  = ['Rho']
samicol  = ['edens']

In [18]:
gitmcol + samicol

['Rho', 'edens']

In [19]:
# Just want to make an example time_list. 
# Can & should be changed in the future.


times = []
t = t_05
max_dtime = t_05 + timedelta(days=7)
while t < max_dtime:
    times.append(t)
    t+=timedelta(minutes=5)
times = np.array(times)

start_idx = np.argmin(np.abs(times - (t_05 + t_start0)))
end_idx = np.argmin(np.abs(times - (t_05 + t_end0)))

In [20]:
data = {}
data_dirs_dict = {}

for mn in dirs.keys():
    for path in os.listdir(dirs[mn]):
        data_dirs_dict[mn+'-'+path.lower()] = dirs[mn]+path




In [21]:
dirs[mn]

'/glade/u/home/abukowski/scratch/simstorm-20110521/postprocessed/1hr/'

In [22]:
data_dirs_dict

{'may-1hr-full': '/glade/u/home/abukowski/scratch/simstorm-20110521/postprocessed/1hr/1hr-full',
 'may-1hr-ut6': '/glade/u/home/abukowski/scratch/simstorm-20110521/postprocessed/1hr/1hr-ut6',
 'may-1hr-half': '/glade/u/home/abukowski/scratch/simstorm-20110521/postprocessed/1hr/1hr-half'}

In [24]:
pbar=tqdm(total=len(data_dirs_dict), desc='reading in data')


# May files
for mn in dirs.keys():
    for path in os.listdir(dirs[mn]):
        
        # # data[mn+'-'+path.lower()] = 0
        # data[mn+'-'+path.lower() + '-sami'] = auto_read_sami(dirs[mn]+path, cols=samicol,
        #                                              start_idx=start_idx, end_idx=end_idx, use_dask=True)
        # pbar.update()
        if 'Q' not in path:
            data[mn+'-'+path.lower() + '-gitm'] = auto_read_gitm(dirs[mn]+path, cols=gitmcol,
                                                     start_idx=start_idx, end_idx=end_idx, use_dask=True)
        pbar.update()


reading in data:   0%|          | 0/3 [00:20<?, ?it/s]

reading in data: 100%|██████████| 3/3 [13:58<00:00, 287.00s/it]

In [25]:
out_data_dir = '/glade/u/home/abukowski/scratch/tmp/rho_edens_datasets'

In [26]:
data.keys()

dict_keys(['may-1hr-full-gitm', 'may-1hr-ut6-gitm', 'may-1hr-half-gitm'])

In [27]:
for file in data.keys():
    data[file].to_netcdf(os.path.join(out_data_dir, file),
        encoding={'time': {"dtype": "float"}})


In [28]:
print('done all!!')

done all!!
